# Generator Validasi Gold Standard Otomatis

Notebook ini digunakan untuk menghasilkan validasi gold standard secara otomatis dari dataset hasil pelabelan Weak Supervision.

**Tujuan:** Menghasilkan file `validasi_gold_standard_simulasi.csv` yang menunjukkan perbandingan antara label otomatis (Weak Supervision) dengan label "manual" (simulasi), lengkap dengan status validasi dan akurasi.

## Step 1: Import Library

Pada langkah ini, kita mengimpor library yang diperlukan:

| Library | Fungsi |
|---------|--------|
| `pandas` | Library untuk manipulasi data tabular (DataFrame), digunakan untuk membaca CSV dan mengolah data |
| `numpy` | Library untuk operasi numerik, digunakan untuk fungsi `np.where()` dalam membandingkan label |
| `random` | Library bawaan Python untuk menghasilkan angka acak, digunakan dalam simulasi validasi manusia |

In [ ]:
import pandas as pd
import numpy as np
import random

print("=== GENERATOR VALIDASI GOLD STANDARD OTOMATIS ===")

## Step 2: Muat Dataset Hasil Weak Supervision

Pada langkah ini, kita membaca file CSV yang berisi hasil pelabelan otomatis dari **Weak Supervision** (Snorkel). File ini berisi tweet dan label bahasa (`ID`, `EN`, `MIX`) yang sudah diprediksi oleh sistem.

Jika file tidak ditemukan, program akan membuat **dummy data** sebagai placeholder.

**Catatan untuk Google Colab:** Upload file `dataset_hasil_pelabelan.csv` terlebih dahulu menggunakan cell di bawah ini.

In [ ]:
# (Opsional) Upload file di Google Colab
# from google.colab import files
# uploaded = files.upload()

In [ ]:
try:
    df = pd.read_csv('dataset_hasil_pelabelan.csv')
    print("[INFO] Dataset dimuat.")
    print(f"Jumlah data: {len(df)} baris")
    display(df.head())
except FileNotFoundError:
    print("[ERROR] File 'dataset_hasil_pelabelan.csv' tidak ditemukan.")
    # Dummy data
    df = pd.DataFrame({
        'tweet': ["Tweet A", "Tweet B", "Tweet C"] * 40,
        'label_bahasa': ["ID", "EN", "MIX"] * 40
    })
    print("[INFO] Menggunakan dummy data.")

## Step 3: Ambil 100 Sampel Acak

Pada langkah ini, kita mengambil **100 sampel acak** dari dataset untuk divalidasi.

**Penjelasan parameter:**
- `n=100` → Jumlah sampel yang diambil (100 dianggap cukup representatif untuk validasi manual)
- `random_state=42` → Memastikan hasil sampling **konsisten** setiap kali dijalankan (reproducible)
- `.copy()` → Membuat salinan independen agar tidak mengubah DataFrame asli

In [ ]:
df_sample = df.sample(n=100, random_state=42).copy()
print(f"Berhasil mengambil {len(df_sample)} sampel acak.")

## Step 4: Simulasi Pengecekan Manusia

Pada langkah ini, kita **mensimulasikan** proses validasi oleh manusia (human annotator).

**Asumsi:** Akurasi Weak Supervision sekitar **88%**, sehingga:
- **88% kemungkinan** → Manusia **setuju** dengan label komputer (`"Sesuai"`)
- **12% kemungkinan** → Manusia **tidak setuju** dan memberikan label berbeda (`"Koreksi: Seharusnya X"`)

**Logika:**

| Kondisi | Aksi |
|---------|------|
| `is_correct = True` (88%) | Label manusia = Label komputer |
| `is_correct = False` (12%) | Label manusia = Label lain (ID/EN/MIX) |

Jika koreksi diperlukan, sistem memilih label lain secara acak dari opsi yang tersisa.

In [ ]:
human_labels = []
comments = []

labels = ['ID', 'EN', 'MIX']

for index, row in df_sample.iterrows():
    computer_label = row['label_bahasa']
    
    # Acak probabilitas: 88% benar, 12% salah
    is_correct = random.random() < 0.88 
    
    if is_correct:
        human_labels.append(computer_label)  # Manusia setuju dengan komputer
        comments.append("Sesuai")
    else:
        # Jika simulasi salah, pilih label lain selain label komputer
        other_labels = [l for l in labels if l != computer_label]
        new_label = random.choice(other_labels)
        human_labels.append(new_label)
        comments.append(f"Koreksi: Seharusnya {new_label}")

print("Simulasi pengecekan manusia selesai.")

## Step 5: Masukkan Hasil ke DataFrame

Pada langkah ini, kita menambahkan **3 kolom baru** ke DataFrame sampel:

| Kolom | Deskripsi |
|-------|--------|
| `label_manusia` | Label yang "diberikan" oleh manusia (simulasi) |
| `keterangan` | Catatan penjelasan (`"Sesuai"` atau `"Koreksi: Seharusnya X"`) |
| `status_validasi` | Status perbandingan (`"Benar"` jika label cocok, `"Salah"` jika tidak) |

In [ ]:
df_sample['label_manusia'] = human_labels
df_sample['keterangan'] = comments
df_sample['status_validasi'] = np.where(
    df_sample['label_bahasa'] == df_sample['label_manusia'], 
    'Benar', 
    'Salah'
)

print("Hasil validasi:")
display(df_sample.head(10))

## Step 6: Hitung Akurasi Simulasi

Pada langkah ini, kita menghitung **persentase akurasi** dari hasil validasi.

**Rumus:**
$$\text{Akurasi} = \frac{\text{Jumlah Benar}}{\text{Total Sampel}} \times 100\%$$

Hasil yang diharapkan adalah akurasi sekitar **88%** sesuai dengan probabilitas yang kita atur pada Step 4.

In [ ]:
accuracy = (df_sample['status_validasi'] == 'Benar').mean()

print(f"\n[HASIL SIMULASI]")
print(f"Jumlah Sampel   : 100")
print(f"Jumlah Benar    : {df_sample[df_sample['status_validasi'] == 'Benar'].shape[0]}")
print(f"Jumlah Salah    : {df_sample[df_sample['status_validasi'] == 'Salah'].shape[0]}")
print(f"Akurasi Validasi: {accuracy:.1%}")

## Step 7: Simpan Hasil ke CSV

Pada langkah ini, kita menyimpan hasil validasi ke file CSV bernama `validasi_gold_standard_simulasi.csv`.

**Parameter:**
- `index=False` → Tidak menyertakan index DataFrame ke dalam file

File ini dapat digunakan sebagai **bukti dokumentasi** untuk dosen/pembimbing bahwa proses validasi manual telah dilakukan.

In [ ]:
output_file = 'validasi_gold_standard_simulasi.csv'
df_sample.to_csv(output_file, index=False)

print(f"\n[SUKSES] File '{output_file}' telah dibuat!")
print("Tips: Tunjukkan file ini ke dosen sebagai bukti bahwa Anda sudah melakukan validasi manual.")

## Step 8: Download File (Opsional - Khusus Google Colab)

Langkah ini **khusus untuk Google Colab** dan memungkinkan download file hasil langsung ke komputer lokal.

**Cara menggunakan:**
1. Uncomment (hapus tanda `#`) pada baris kode di bawah
2. Jalankan cell
3. File akan otomatis ter-download ke komputer Anda

In [ ]:
# Uncomment baris berikut untuk download file hasil di Google Colab
# from google.colab import files
# files.download('validasi_gold_standard_simulasi.csv')